In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

![man-looking-online-coronavirus-updates_52683-49025.jpg](attachment:50672340-5192-4dbf-ade6-5ef32149aacd.jpg)

# Summary🖋️
Fake news refers to deliberately fabricated or misleading information presented as factual news. It often aims to deceive readers or viewers for various purposes, such as influencing opinions, spreading propaganda, or generating revenue through clicks. Detecting fake news is crucial in combating misinformation and maintaining the integrity of journalism and public discourse. Deep learning techniques, such as natural language processing and neural networks, can be effective in identifying patterns and features indicative of fake news, helping to automate the process of verification and fact-checking.

# What you will find in this notebook📒-
1. Utilizing **LSTM (Long Short-Term Memory) RNN** for Fake News Detection.
2. Feature embedding is constructed before LSTM, facilitating data representation.
3. Preprocessing includes **stemming
SnowballStemmer
regex** using **stemming
SnowballStemmer
regex** and **stemming
SnowballStemmer
regex** cleaning.
4. One-hot encoding by **Keras** prepares textual data for LSTM input.
5. Adam optimizer and **binary_crossentropy** loss function are employed during model compilation.
6. Model performance evaluation includes **classification report and confusion matrix**.
7. Determination of the optimal **threshold value** for prediction refinement.

# Let's Dive In🏊‍

In [2]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from keras import Sequential
from keras.layers import Embedding, Dense, LSTM
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
import nltk
from nltk.stem.snowball import SnowballStemmer
import regex as re
from nltk.tokenize import sent_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords


In [3]:
# download some packages
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [7]:
# datasets
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [ ]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [8]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [9]:
# label them seperately
df_true['status'] = 0
df_fake['status'] = 1

In [10]:
# merge and remove unnecessary columns
df = pd.concat([df_true,df_fake])
df.drop(['subject','text','date'],axis=1,inplace=True)

* Since we are going to build model only based on the title feature, hence dropped text, date , subject

In [11]:
# let's blend the smoothie
random_indexes = np.random.randint(0,len(df),len(df))
df = df.iloc[random_indexes].reset_index(drop=True)

## Text analysis

In [12]:
pd.set_option('display.max_colwidth', 500)
random = np.random.randint(0,len(df),20)
df.iloc[random]

,title,status
10755,WOW! “N” Word Used On Walmart Website To Describe Color Of Wig,1
42221,OVER 100 COLLEGES WILL HOLD SEPARATE GRADUATION Ceremonies For Gays As Part Of A “Cultural Celebration”,1
25804,U.S. Embassy move can be done in 'two minutes': Jerusalem mayor,0
10192,U.S sanctions North Koreans for 'flagrant' rights abuse,0
6594,"Venezuela slams Canada sanctions, says Ottawa submitting to Trump",0
14464,Queen to invite Trump to Britain for state visit: Sunday Times,0
16171,WHY IS THE PRESENCE OF RUSSIAN AIRCRAFT FIGHTER JETS NEAR U.S. SHORES INCREASING?,1
37936,Senator McCaskill drafts bill in response to Allergan patent maneuver,0
7040,Woman Raped By Trump At 13 May Have Been Terrorized Into Dropping Suit,1
2589,Family’s Dog Poisoned By Flint’s Toxic Lead Water,1


* work needs to be done on data before feeding to neural network-
1. Remove punctuations eg ""
2. Convert uppercase to lowercase
3. No need to apply stemming, otherwise it will just shorten the word unnecessary
4. Apply lemmatization
5. Remove all the stopwords
6. Finally make vocabulary after completion of 5 steps

In [13]:
# Null values
df.isnull().sum()

title     0
status    0
dtype: int64

In [14]:
# longest sentence length
def longest_sentence_length(text):
  return len(text.split())

df['maximum_length'] = df['title'].apply(lambda x : longest_sentence_length(x))
print('longest sentence having length -')
max_length = max(df['maximum_length'].values)
print(max_length)

longest sentence having length -
41


In [15]:
# Text cleaning
text_cleaning = "\b0\S*|\b[^A-Za-z0-9]+"

def preprocess_filter(text, stem=False):
  text = re.sub(text_cleaning, " ",str(text.lower()).strip())
  tokens = []
  for token in text.split():
    if token not in stop_words:
      if stem:
        stemmer = SnowballStemmer(language='english')
        token = stemmer.stem(token)
      tokens.append(token)
  return " ".join(tokens)

* The regular expression pattern removes non-alphanumeric characters and digits, enhancing text cleanliness for subsequent analysis.
* Text preprocessing function standardizes text by converting it to lowercase, removing stop words, and optionally applying stemming, facilitating effective natural language processing.
* Stemming simplifies words to their root forms, aiding in capturing underlying semantic meanings and improving search accuracy.
* Removal of stop words reduces noise in text data, focusing on informative words and enhancing the quality of subsequent analysis.

In [16]:
# Word embedding with pre padding
def one_hot_encoded(text,vocab_size=5000,max_length = 40):
    hot_encoded = one_hot(text,vocab_size)
    return hot_encoded

In [17]:
# word embedding pipeline
def word_embedding(text):
    preprocessed_text=preprocess_filter(text)
    return one_hot_encoded(preprocessed_text)

In [18]:
# Creating NN Model
embedded_features = 40
model = Sequential()
model.add(Embedding(5000,embedded_features,input_length = max_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy',optimizer= 'adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 41, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


* The model utilizes a vocabulary size of 5000, reflecting the extensive nature of the dataset and the need to handle a wide range of words effectively.
* By embedding input tokens into 40-dimensional vectors, the model captures nuanced semantic relationships, crucial for understanding the complex language patterns present in the dataset.
* Leveraging an LSTM layer with 100 units, the model effectively learns from the extensive sequential data, ensuring it captures long-term dependencies and context effectively.
* With a final dense layer employing a sigmoid activation function, the model delivers binary classification predictions, adeptly classifying the vast and varied dataset with accuracy.

In [19]:
# One hot encoded title
one_hot_encoded_title =df['title'].apply(lambda x : word_embedding(x)).values

In [20]:
# padding to make the size equal of the sequences
padded_encoded_title = pad_sequences(one_hot_encoded_title,maxlen=max_length,padding = 'pre')

In [21]:
# Splitting
X = padded_encoded_title
y = df['status'].values
y = np.array(y)

# shapes
print(X.shape)
print(y.shape)

(44898, 41)
(44898,)


In [22]:
# shape and size
print('X shape {}'.format(X.shape))
print('y shape {}'.format(y.shape))

X shape (44898, 41)
y shape (44898,)


In [23]:
# Splitting into training, testing
X_train,X_test,y_train,y_test = train_test_split(X,y, random_state = 42)

# Shape and size of train and test dataset
print('X train shape {}'.format(X_train.shape))
print('X test shape {}'.format(X_test.shape))
print('y train shape {}'.format(y_train.shape))
print('y test shape {}'.format(y_test.shape))

X train shape (33673, 41)
X test shape (11225, 41)
y train shape (33673,)
y test shape (11225,)


In [ ]:
# Model training
# training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=15,batch_size=64)

Epoch 1/15
527/527 [==============================] - 40s 70ms/step - loss: 0.2009 - accuracy: 0.9133 - val_loss: 0.1198 - val_accuracy: 0.9556
Epoch 2/15
527/527 [==============================] - 38s 71ms/step - loss: 0.0717 - accuracy: 0.9748 - val_loss: 0.1093 - val_accuracy: 0.9632
Epoch 3/15
527/527 [==============================] - 38s 73ms/step - loss: 0.0411 - accuracy: 0.9861 - val_loss: 0.1195 - val_accuracy: 0.9649
Epoch 4/15
527/527 [==============================] - 38s 72ms/step - loss: 0.0242 - accuracy: 0.9926 - val_loss: 0.1777 - val_accuracy: 0.9585
Epoch 5/15
527/527 [==============================] - 37s 70ms/step - loss: 0.0182 - accuracy: 0.9939 - val_loss: 0.1785 - val_accuracy: 0.9643
Epoch 6/15
527/527 [==============================] - 38s 72ms/step - loss: 0.0116 - accuracy: 0.9962 - val_loss: 0.1689 - val_accuracy: 0.9671
Epoch 7/15
527/527 [==============================] - 37s 69ms/step - loss: 0.0103 - accuracy: 0.9969 - val_loss: 0.1876 - val_accuracy:

## Evaluation

In [ ]:
# setting threshold value
def best_threshold_value(thresholds:list,X_test):
    accuracies = []
    for thresh in thresholds:
        ypred =model.predict(X_test)
        ypred = np.where(ypred> thresh,1,0)
        accuracies.append(accuracy_score(y_test,ypred))
    return pd.DataFrame({
        'Threshold': thresholds,
        'Accuracy' : accuracies
    })

In [ ]:
best_threshold_value([0.4,0.5,0.6,0.7,0.8,0.9], X_test)

351/351 [==============================] - 1s 2ms/step


,Threshold,Accuracy
0,0.4,0.965880
1,0.5,0.965969
2,0.6,0.966325
3,0.7,0.966860
4,0.8,0.966860
5,0.9,0.966771


* Not much difference in accuray
* But the most suitable threshold value we have got is **0.4**

In [ ]:
# Predictino value at threshold 0.4
y_pred = model.predict(X_test)
y_pred = np.where(y_pred >0.4, 1, 0)

351/351 [==============================] - 1s 2ms/step


In [ ]:
# Confusion matrix
print('Confusion matrix')
print(confusion_matrix(y_pred,y_test))
print('----------------')
print('Classification report')
print(classification_report(y_pred,y_test))

Confusion matrix
[[5045  153]
 [ 230 5797]]
----------------
Classification report
              precision    recall  f1-score   support

           0       0.96      0.97      0.96      5198
           1       0.97      0.96      0.97      6027

    accuracy                           0.97     11225
   macro avg       0.97      0.97      0.97     11225
weighted avg       0.97      0.97      0.97     11225



* The model performs well in both classes, with high precision, recall, and F1-score, suggesting robustness in classification.
* There is no significant imbalance in performance between the two classes, as evidenced by similar metrics for both classes.
* The model's overall performance is excellent, achieving high accuracy on the dataset.

## Predictions

In [ ]:
# input generator
def prediction_input_processing(text):
    encoded = word_embedding(text)
    padded_encoded_title = pad_sequences([encoded],maxlen=max_length,padding = 'pre')
    output = model.predict(padded_encoded_title)
    output = np.where(0.4>output,1,0)
    if output[0][0] == 1:
        return 'Yes this News is fake'
    return 'No, It is not fake'


In [ ]:
# predictions
prediction_input_processing('Americans are more concerned over Indians fake open source contribution')

1/1 [==============================] - 0s 20ms/step


'Yes this News is fake'

In [ ]:
news = 'Trump Just Sent Michelle Obama a Bill She will Never Be able to pay in her lifetime'
prediction_input_processing(news)

1/1 [==============================] - 0s 19ms/step


'No, It is not fake'

__________________________________________________________

# आभारं तव कालाय। भगवान् त्वां धन्यतां प्रददातु।🙏🙏

________________________________________________________________